# FSLogix AppMasking mit PowerShell
## Andreas Nick 2021

Basis für diese Seite ist Visual Studio Code mit dem Interactiven Notebook und der Erweiterung Jupyter

## Quellen

>1. [VisualStudio Code](https://code.visualstudio.com/)
>2. Visual Studio Code Erweiterung: [.NET Interactive Notebooks](https://marketplace.visualstudio.com/items?itemName=ms-dotnettools.dotnet-interactive-vscode)
>3. Visual Studio Code Erweiterung: [Jupyter](https://marketplace.visualstudio.com/items?itemName=ms-toolsai.jupyter)
>4. .NET 5 SDK (x86): [.NET 5](https://dotnet.microsoft.com/download/dotnet/5.0)
>5. FSlogix [FSLogix](https://aka.ms/fslogix/download)

In [ ]:
#Download FSLogix
Invoke-WebRequest -Uri https://aka.ms/fslogix/download -OutFile "$ENV:USERPROFILE\Desktop\FSLogixApps.zip" 

## Installieren
Aus dem Archiv FSLogixApps die Elemente FSLogixAppsRuleEditorSetup.exe und FSLogixAppsSetup.exe installieren

## Einige Pakete zum Testen

In [ ]:
#App-V Macht etwas Probleme Enable-AppV
& powershell.exe -Command {Enable-AppV}
& powershell.exe -Command {Get-ChildItem C:\temp\AppVPakete_2021\*.appv -Recurse| Add-AppVClientPackage | Publish-AppVClientPackage -Global}
#& powershell.exe -Command {Get-AppvClientPackage | unpublish-AppVClientPackage -global}

## Basis Modul - FSLogix

In [ ]:
Install-Module -Name FSLogix.PowerShell.Rules -Scope CurrentUser -Confirm:$false -Force
Import-Module -Name FSLogix.PowerShell.Rules
Get-Command -Module FSLogix.PowerShell.Rules


## FSLogix Rule Module Befehlsübersicht

| CommandType |         Name         |
| :---------: | :------------------: |
|  Function   |  Add-FslAssignment   |
|  Function   |     Add-FslRule      |
|  Function   | Compare-FslFilePath  |
|  Function   | Compare-FslRuleFile  |
|  Function   |  Get-FslAssignment   |
|  Function   |  Get-FslLicenseDay   |
|  Function   |     Get-FslRule      |
|  Function   | Remove-FslAssignment |
|  Function   |    Remove-FslRule    |
|  Function   |  Set-FslAssignment   |
|  Function   |  Set-FslLicenseDay   |
|  Function   |     Set-FslRule      |

## Es wird Zeit, etwas aufzuräumen

In [ ]:
Import-Module -Name FSLogix.PowerShell.Rules
$FSRulePath = "C:\Program Files\FSLogix\Apps\Rules\"

In [ ]:
Import-Module -Name FSLogix.PowerShell.Rules
$FSRulePath = "C:\Program Files\FSLogix\Apps\Rules\"

# Die FRX Datei enthält ein Regelwerg
#Set überschreibt die Datei oder legt eine neue Datei an
Set-FslRule -Path "$FSRulePath\Desktop.fxr" -FullName "C:\Users\Public\desktop" -HidingType FolderOrKey 

# Die FRA Datei die Berechtigungen
Set-FslAssignment  -Path "$FSRulePath\Desktop.fxa" -UserName 'Administrator' -RuleSetApplies:$true

## Etwas mehr aufräumen?

In [ ]:
#Add fügt einen Eintrag hinzu
Add-FslRule -Path "$FSRulePath\Desktop.fxr" -FullName "$ENV:USERPROFILE\Desktop" -HidingType FolderOrKey 

## Oder vielleicht doch nicht?

In [ ]:
#Set-FslAssignment  -Path "$FSRulePath\Desktop.fxa" -UserName 'Administrator' -RuleSetApplies:$false
#Netz-Weise
Add-FslAssignment  -Path "$FSRulePath\Desktop.fxa" -UserName 'Student' -RuleSetApplies:$false

In [ ]:
# App im Startmenü verstecken

if(Test-Path  "$FSRulePath\AdminTools.fxr") {Remove-Item "$FSRulePath\AdminTools.fxr"}
Get-ChildItem "$env:ALLUSERSPROFILE\Microsoft\Windows\Start Menu\Programs\Administrative Tools\*.lnk" -Recurse | Foreach-Object {Add-FslRule -Path "$FSRulePath\AdminTools.fxr" -FullName "$_" -HidingType FileOrValue}

#Set-FslRule -Path "$FSRulePath\AdminTools.fxr" -FullName 'C:\ProgramData\Microsoft\Windows\Start Menu\Programs\Administrative Tools\Component Services.lnk' -HidingType FileOrValue



In [ ]:
#Und einschalten
#Set-FslAssignment  -Path "$FSRulePath\AdminTools.fxa" -UserName 'Administrator' -RuleSetApplies:$true
Set-FslAssignment  -Path "$FSRulePath\AdminTools.fxa" -UserName 'Student' -RuleSetApplies:$true
#Das Ganze funktioniert natürlich auch für weitere Gruppen. Dann hier beispielsweise mit Add-FSLAssignment eine weitere Gruppe einfügen
#Add-FslAssignment  -Path "$FSRulePath\AdminTools.fxa" -UserName 'Benutzer' -RuleSetApplies:$true
Add-FslAssignment  -Path "$FSRulePath\AdminTools.fxa" -UserName 'Users' -RuleSetApplies:$true

In [ ]:
# Last Write wins!
Add-FslAssignment  -Path "$FSRulePath\AdminTools.fxa" -UserName 'Administrator' -RuleSetApplies:$false
Add-FslAssignment  -Path "$FSRulePath\AdminTools.fxa" -UserName 'Student' -RuleSetApplies:$false

## Und noch einen Ordner und einen RegKey verschwinden lassen

In [ ]:
Set-FslRule -Path "$FSRulePath\Folder.fxr" -FullName "C:\temp\AppVPakete_2021" -HidingType FolderOrKey
Add-FslRule -Path "$FSRulePath\Folder.fxr" -FullName "HKEY_LOCAL_MACHINE\SOFTWARE\Mozilla" -HidingType FolderOrKey
#Netz-Weise
Add-FslAssignment  -Path "$FSRulePath\Folder.fxa" -UserName 'Student' -RuleSetApplies:$true

In [ ]:
#Und wieder Da - diesmal anders
remove-Item "$FSRulePath\Folder.fxr"
Remove-Item "$FSRulePath\Folder.fxa" 

## Andere Möglichkeiten
Eine der wichtigsten Funktionen ist das Verstecken von Registrierungsschlüsseln, Ordnern und Dateien
FSLogix Regeln können aber noch mehr:
- Umleiten von Dateien und Ordnern
- 

## Umleiten in VHD

In [ ]:
#Wenn Hyper-V installiert ist
New-VHD -Path c:\temp\Base.vhdx -SizeBytes 1GB

In [ ]:
Add-FslRule -Path "$FSRulePath\ToVHD.fxr" -DiskFile "c:\temp\Base.vhdx" -FullName "$env:USERSPROFILE\Favorites" 

In [ ]:
# Order Umleitung
if(-not (Test-Path 'c:\Temp\RedFolder')) {New-Item 'c:\Temp\RedFolder' -ItemType Directory -Force}
Set-FslRule -Path "$FSRulePath\redirect.fxr"  -FullName "$env:USERPROFILE\Favorites\"  -RedirectType FolderOrKey -RedirectDestPath "c:\Temp\RedFolder" -CopyObject
Set-FslAssignment -Path "$FSRulePath\redirect.fxa" -UserName 'Student' -RuleSetApplies:$true


## Cleanup

In [ ]:
Get-ChildItem "$FSRulePath" | % {Remove-Item $_ }
& powershell.exe -Command {Get-AppVclientPackage | Unpublish-AppvclientPackage -Global}
& powershell.exe -Command {Get-AppVclientPackage -all | Remove-AppVClientPackage}


PackageId            : 0d981e00-6be8-4d9e-85a8-320b3660770d
VersionId            : 4872e878-8022-478a-92a1-e904e3f67b32
Name                 : Greenshot_x86_German_1.2.10.6
Version              : 0.0.0.1
IsPublishedToUser    : False
UserPending          : False
IsPublishedGlobally  : False
GlobalPending        : False
InUse                : False
InUseByCurrentUser   : False
HasAssetIntelligence : True

PackageId            : 00b87000-1edc-46a6-93ea-f3a5af851ef1
VersionId            : 440b145d-ae46-4881-a5a3-0c115b80a783
Name                 : sqavw13_wxp_de
Version              : 0.0.0.1
IsPublishedToUser    : False
UserPending          : False
IsPublishedGlobally  : False
GlobalPending        : False
InUse                : False
InUseByCurrentUser   : False
HasAssetIntelligence : True

PackageId            : 48927726-3a0c-4fcc-8c73-5f9b8856cf9d
VersionId            : 9891af64-fed9-457c-87cd-0a9d750e2aca
Name                 : PaintDotNet_x64_4.216.7781.39227
Version              : 0